In [34]:
import subprocess
import mysql.connector
import os

In [40]:
mysql_conn = mysql.connector.connect(
    host='34.46.191.121',
    port=3306,
    user='root',
    database='auditpulse',
    password=os.getenv('MYSQL_GCP_PASS')
)

mysql_cursor = mysql_conn.cursor()

query = """
SELECT COUNT(*) FROM runs 
WHERE status='queued'
"""
mysql_cursor.execute(query)

In [ ]:
result = mysql_cursor.fetchone()
result
print(f'Count: {result}')

(0,)

In [49]:
count = 1000
per_request_time_est = 15*60
per_instance_request = 2
threshold = 0.2
max_time_allowed = 6*60*60
total_time_est = count*per_request_time_est//per_instance_request
total_time_est*=1+threshold
total_time_est = int(total_time_est)
total_time_est = min(total_time_est, max_time_allowed)
print(f'Total time estimate: {total_time_est}s')

Total time estimate: 21600s


In [33]:
project_id = 'auditpulse'  # From your YAML namespace
region = 'us-central1'       # From your YAML labels
job_name = 'auditpulse-app'  # From your YAML metadata
image = "us-east1-docker.pkg.dev/auditpulse/auditpulse-images/auditpulse-app:latest"

# gcloud run jobs update auditpulse-app ^
#   --image us-east1-docker.pkg.dev/auditpulse/auditpulse-images/auditpulse-app:latest ^
#   --region us-central1 ^
#   --project auditpulse ^
#   --cpu 4000m ^
#   --memory 2Gi ^
#   --task-timeout 3600s ^
#   --max-retries 1 ^
#   --service-account 853525367358-compute@developer.gserviceaccount.com ^
#   --set-secrets SERPER_API_KEY=SERPER_API_KEY:latest,AGENTOPS_API_KEY=AGENTOPS_API_KEY:latest,MYSQL_GCP_PASS=MYSQL_GCP_PASS:latest ^
#   --execute-now

cmd = [
    "gcloud", "run", "jobs", "update", job_name,
    "--image", image,
    "--region", region,
    "--project", project_id,
    "--cpu", "4000m",
    "--memory", "2Gi",
    "--task-timeout", "600s",
    "--max-retries", "1",
    "--tasks", "2",
    "--service-account","853525367358-compute@developer.gserviceaccount.com",
    "--set-secrets", "SERPER_API_KEY=SERPER_API_KEY:latest,AGENTOPS_API_KEY=AGENTOPS_API_KEY:latest,MYSQL_GCP_PASS=MYSQL_GCP_PASS:latest",
    "--execute-now" 
]
    
# cmd_string = f"gcloud run jobs update {job_name} --image {image} --region {region} --project {project_id} --cpu 4000m --memory 2Gi --task-timeout 3600s --max-retries 1 --service-account 853525367358-compute@developer.gserviceaccount.com --set-secrets 'SERPER_API_KEY=SERPER_API_KEY:latest,AGENTOPS_API_KEY=AGENTOPS_API_KEY:latest,MYSQL_GCP_PASS=MYSQL_GCP_PASS:latest' --execute-now"
    
# result = subprocess.run(cmd_string, shell=True, check=True, capture_output=True)
result = subprocess.run(cmd, shell=True, check=True, capture_output=True)